In [64]:
input_fasta_file_path = r'C:\Users\dell\Desktop\basecalls_assignment\input_fasta.fasta'
bed_filename = 'output.bed'


# opening the input fasta file in read mode
fr = open(input_fasta_file_path, 'r')

# opening the output bed file in write mode
fw_bed = open(bed_filename, 'w')

# dict storing all the GIds from which subsequence has to be fetched along with the start and end coordinates
gids_dict = {'661921128': '100-200', '1779817629': '1200-1500', '1839253191': '650-1550'}

# this dictionary will store the sequence ids as keys and the subsequence as its values
final_dict = {}

# this variable is used to fetch entire sequences as a single string from the multi fasta file from which we fetch the subsequence 
seq = ""

# reading multi fasta file line by line
for line in fr:
    line = line.rstrip('\n') # removing '\n' from the line
    if '>' in line: # if line has '>' sign
        if seq != "": # this condition checks if seq is not equal to an empty string
            coords = gids_dict[gid].split('-')
            start_coord = int(coords[0]) - 1
            end_coord = int(coords[1])
            final_dict[gid] = seq[start_coord: end_coord] # adding sub sequence to the final dict with id as the key
            seq = "" # again converting seq to an empty string
            
            # writing contents to the BED file
            # BED format used 0 based coordinate system for start coordinates and 1 based for end coordinate
            # which is the same way in which we specify a range in python
            bed_line = gid + '\t' + str(start_coord) + '\t' + str(end_coord) + '\n'
            fw_bed.write(bed_line)

        gid = line.split('|')[1] # extracting only id from the header line
        
        if gid in gids_dict: # check if the id is present in the dict of our required ids
            found = 1 # setting the value of found to 1
            continue # continue the loop
            
        else: # if the id is not found in the list of our required ids
            found = 0 # setting the value of found to 0
            
    # this code will execute only when found = 1
    # that is when we found any of the required ids in the multi fasta file
    if found == 1:
        seq += line # we start building the sequence by adding line to the seq variable
        
# closing the input file
fr.close()

# closing the output BED file
fw_bed.close()

print('Output BED file has been successfully created')

Output BED file has been successfully created


In [65]:
final_dict

{'661921128': 'AATATTAAACCTCTTTCTGATGGTCTCTATAAACGATTGATTTTTCTTACCCTCTAGAGTTTCCTACGGTCGTGGGTCACACATTTTTTTCTAGACACTAA',
 '1779817629': 'CAAATCTTATCATTGGTCGGTGTTTGGTCTTGCTTTGTGACTTTGATAATAACATCGATTCCCATATGATGTTTGTTTTCTTCTTCAGTACACGAGGATGAAGATTGTTGAAGACTAGTAGGCATAGCAGCTGCCACTAGGCACATGCATGCCAGGACAATATATTGTTTCATGATTGCTATTGATTGATTACTGTTCTAGATGATTCTACTTTCTTACCATATAATAAATTAGAATATATTTTCTACTTTTACGAGAAATTAATTATTGTATTTATTATTTATAGGTAAAAAAACTTACT',
 '1839253191': 'TTTTGTTGAATCGATGAGTGAAGTATCATCGGTTGCACCTTCAGATGCCGATCCGTCGACATACTTGAATCCATCCTTGACTTCAAGTTCAGATGATTCCTCACACATGTCTCCGATACGTACGCTAAACTCTAGGTTCTTGACACATTTTGTATCAACGATCGTTGAACCGATGATATCTTTGTAACTCACTTTCTTATGTGAGATGTTAGACCCAAGTACTGGATGGGTCTTGATGTCACTGTCTTTCTCTTCTTCGCTACATCTGATGTCGATAGACATCTCACAGTCTTTGATCATAGCCAGAGCTTCTTCACGCGTGATCGCGGGAGAGTCCTTACCTTGTCCCGGTGACACGCTGGACAATCTAGTATTCACAGTGTTTCCATCAGAGGATTCGGAGATGGATGAAATCTTTGGGCATTTGGTGAATCCAAAGTTCATGTTAAGACCCGCACCGACGATAGTGTAATAAGTGGTGGGATCTCCTTTTACAACTTCTTCGGATACCTCATCATCTTCGGTCTCTGTAACTTCCGTTACGG

In [66]:
# reverse complementing the extracted subsequences

def rev_comp(seq):
    comp_bases = {'A':'T', 'T':'A', 'G':'C', 'C':'G'}
    complement = "" # this will store the complement of a sequence
    for i in range(len(seq)):
        complement += comp_bases[seq[i]]
    rev_comp = complement[::-1] # reversing the complement sequence
    return rev_comp

final_dict_rev_comp = {}
for gid in final_dict:
    final_dict_rev_comp[gid] = rev_comp(final_dict[gid])

In [67]:
final_dict_rev_comp

{'661921128': 'TTAGTGTCTAGAAAAAAATGTGTGACCCACGACCGTAGGAAACTCTAGAGGGTAAGAAAAATCAATCGTTTATAGAGACCATCAGAAAGAGGTTTAATATT',
 '1779817629': 'AGTAAGTTTTTTTACCTATAAATAATAAATACAATAATTAATTTCTCGTAAAAGTAGAAAATATATTCTAATTTATTATATGGTAAGAAAGTAGAATCATCTAGAACAGTAATCAATCAATAGCAATCATGAAACAATATATTGTCCTGGCATGCATGTGCCTAGTGGCAGCTGCTATGCCTACTAGTCTTCAACAATCTTCATCCTCGTGTACTGAAGAAGAAAACAAACATCATATGGGAATCGATGTTATTATCAAAGTCACAAAGCAAGACCAAACACCGACCAATGATAAGATTTG',
 '1839253191': 'TGCGATATACATAAACTGATCACTAATTCCCAGAATCCCACCCACTTATAGTAAGTTTTTTTTACCTATAAATAATAAATACAATAATTAATTTCTCGTAAAAGTAGAAAATATATTCTAATTTATTATATGGTAAGAAAGTAGAATCATCTAGAACAGTAATCAATCAATAGCAATCATGAAACAATATATTGTCCTGGCATGCATGTGCCTAGTGGCAGCTGCTATGCCTACTAGTCTTCAACAATCTTCATCCTCGTGTACTGAAGAAGAAAACAAACATCATATGGGAATCGATGTTATTATCAAAGTCACAAAGCAAGACCAAACACCGACCAATGATAAGATTTGTCAATCCGTAACGGAAGTTACAGAGACCGAAGATGATGAGGTATCCGAAGAAGTTGTAAAAGGAGATCCCACCACTTATTACACTATCGTCGGTGCGGGTCTTAACATGAACTTTGGATTCACCAAATGCCCAAAGATTTCATCCATCTCCGAATCCTCTGATGGAAACACTGTGAATACTAGATTGTCCAGCG

In [107]:
# now adding back the reverse complemented sub sequences to the original sequences and creating a new output fasta file

output_fasta_filename = 'output.fasta'

# opening the input fasta file in read mode
fr = open(input_fasta_file_path, 'r')

# opening the output fasta file in write mode
fw_fasta = open(output_fasta_filename, 'w')

# reading multi fasta file line by line
seq = ""
header = ""
for line in fr:
    line = line.rstrip('\n') # removing '\n' from the line
    if '>' in line: # if line has '>' sign
        if seq != "":
            if found == 1: # if we are on a requried gid
                coords = gids_dict[gid].split('-')
                start_coord = int(coords[0]) - 1
                end_coord = int(coords[1])
                # replacing original sequence with reverse complemented subsequence within its respective coordinates
                seq = seq[:start_coord] + final_dict_rev_comp[gid] + seq[end_coord:]
            
            fw_fasta.write(header + '\n')
            fw_fasta.write(seq + '\n')
            seq = ""
            
        gid = line.split('|')[1] # extracting only id from the header line
        header = line # storing header for writing in fasta file
        
        if gid in final_dict_rev_comp:
            found = 1
        else:
            found = 0
             
    else:
        seq += line

fr.close()
fw_fasta.close()

print('Output fasta file has been successfully created')

Output fasta file has been successfully created
